In [7]:
import pandas as pd
from tqdm import tqdm
from transformers import pipeline

In [3]:
df = pd.read_csv('wav-label-data.csv')

In [4]:
df.head()

,Unnamed: 0,wav,sentence,label
0,0,Ses01F_impro01_F000,Excuse me.,neu
1,1,Ses01F_impro01_M000,Do you have your forms?,fru
2,2,Ses01F_impro01_F001,Yeah.,neu
3,3,Ses01F_impro01_M001,Let me see them.,fru
4,4,Ses01F_impro01_F002,Is there a problem?,neu


In [8]:
classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

# Function to apply the classifier to each sentence and extract labels and scores
def classify_sentence(sentence):
    model_output = classifier(sentence)
    # Extracting labels and scores
    labels = [output['label'] for output in model_output[0]]
    scores = [output['score'] for output in model_output[0]]
    return labels, scores

# Applying the function to the 'sentence' column
tqdm.pandas(desc="Classifying sentences")
df[['labels', 'scores']] = df['sentence'].progress_apply(lambda x: pd.Series(classify_sentence([x])))


Classifying sentences: 100%|██████████| 10039/10039 [41:54<00:00,  3.99it/s] 


In [9]:
df

,Unnamed: 0,wav,sentence,label,labels,scores
0,0,Ses01F_impro01_F000,Excuse me.,neu,"[neutral, approval, annoyance, confusion, real...","[0.8135871291160583, 0.04733756184577942, 0.03..."
1,1,Ses01F_impro01_M000,Do you have your forms?,fru,"[curiosity, neutral, confusion, annoyance, app...","[0.5902626514434814, 0.42505648732185364, 0.17..."
2,2,Ses01F_impro01_F001,Yeah.,neu,"[neutral, approval, realization, annoyance, ad...","[0.5529206395149231, 0.39811864495277405, 0.02..."
3,3,Ses01F_impro01_M001,Let me see them.,fru,"[neutral, annoyance, approval, anger, curiosit...","[0.9605748653411865, 0.0104795191437006, 0.007..."
4,4,Ses01F_impro01_F002,Is there a problem?,neu,"[curiosity, confusion, neutral, annoyance, app...","[0.5059351921081543, 0.48047932982444763, 0.32..."
...,...,...,...,...,...,...
10034,10034,Ses05M_script03_2_M044,oh! Marry you again? I wouldn't marry you agai...,ang,"[neutral, disapproval, surprise, annoyance, cu...","[0.36445435881614685, 0.1333918273448944, 0.07..."
10035,10035,Ses05M_script03_2_F042,Beast,ang,"[neutral, approval, annoyance, realization, ad...","[0.9613756537437439, 0.013455008156597614, 0.0..."
10036,10036,Ses05M_script03_2_M045,You're a wicked little vampire. And I pray to...,ang,"[admiration, anger, annoyance, caring, desire,...","[0.16776657104492188, 0.16043898463249207, 0.1..."
10037,10037,Ses05M_script03_2_F043,Brute,ang,"[admiration, neutral, approval, realization, d...","[0.9034367203712463, 0.10050792992115021, 0.05..."


In [10]:
df.to_csv('scores_raw.csv')